# **📦 Install Required Packages**

In [ ]:
%%capture
!pip install opencv-python pandas tqdm
!pip install --upgrade ultralytics supervision

# **📦 Required Imports**

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from supervision import Point, LineZone, LineZoneAnnotator, BoxAnnotator, LabelAnnotator, Color
from supervision.detection.core import Detections
from ultralytics import YOLO
from google.colab import files

# **🚀 Load YOLOv8 Model with Fallback**

In [ ]:
def load_yolo_model(model_name="yolov8n.pt"):
    """
    Load YOLOv8 model with automatic download if not found.

    Args:
        model_name (str): Name of YOLO model file.

    Returns:
        YOLO: Loaded YOLO model instance.
    """
    try:
        model = YOLO(model_name)
        return model
    except Exception as e:
        print(f"Model not found locally, downloading {model_name}...")
        model = YOLO(model_name)  # This will automatically download
        return model

# **🎥 Main Video Processing Function**

In [ ]:
def process_video(
    line_x_start=0.05,    # Horizontal start point (left to right ratio)
    line_x_end=0.95,      # Horizontal end point (left to right ratio)
    line_y_position=0.3,  # Fixed vertical position of horizontal line (top to bottom ratio)
    conf_threshold=0.2,   # Detection confidence threshold
    crowd_threshold=10,   # Crowd warning threshold
    imgsz=640             # Model input size
):
    """
    Process a video to detect and track people, count crossings over a horizontal virtual line, and annotate the output.

    Args:
        line_x_start (float): X-coordinate ratio for the start of the horizontal line (0.0 to 1.0, left).
        line_x_end (float): X-coordinate ratio for the end of the horizontal line (0.0 to 1.0, right).
        line_y_position (float): Y-coordinate ratio for the fixed vertical position of the horizontal line (0.0 to 1.0, top).
        conf_threshold (float): Confidence threshold for YOLO model detections.
        crowd_threshold (int): Number of people (in - out) to trigger a crowd warning.
        imgsz (int): Image size for YOLO model processing (e.g., 640).

    Returns:
        None: Outputs an annotated video ('metro_gate_output.mp4') and a CSV file ('counts.csv') with frame-wise counts.
    """
    try:
        # --- Video Upload ---
        print("Please upload a video file (.mp4 or .avi):")
        uploaded = files.upload()
        if not uploaded:
            raise ValueError("No video file uploaded.")
        video_path = next(iter(uploaded))
        if not video_path.lower().endswith(('.mp4', '.avi')):
            raise ValueError("Invalid file format. Please upload a .mp4 or .avi video.")

        # --- Output Setup ---
        os.makedirs('output', exist_ok=True)
        output_video = os.path.join('output', 'metro_gate_output.mp4')
        output_csv = os.path.join('output', 'counts.csv')

        # --- Load YOLO Model ---
        model = load_yolo_model()

        # --- Initialize Video Capture ---
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise ValueError("Failed to open video. Ensure the file is a valid video format.")
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(f"Video: {width}x{height}, FPS: {fps}, Total Frames: {total_frames}")

        # --- Define Horizontal Virtual Line ---
        LINE_START = Point(int(width * line_x_start), int(height * line_y_position))
        LINE_END   = Point(int(width * line_x_end),   int(height * line_y_position))
        print(f"Horizontal Line Start: {LINE_START}, End: {LINE_END}")
        line_counter = LineZone(start=LINE_START, end=LINE_END)

        # --- Initialize Annotators ---
        box_annotator = BoxAnnotator()
        label_annotator = LabelAnnotator()
        line_annotator = LineZoneAnnotator(color=Color(r=173, g=216, b=230))

        # --- Setup Video Writer ---
        writer = cv2.VideoWriter(
            output_video,
            cv2.VideoWriter_fourcc(*'mp4v'),
            fps,
            (width, height)
        )
        if not writer.isOpened():
            raise RuntimeError("Failed to initialize video writer.")

        # --- Data Storage ---
        count_data = []
        frame_count = 0

        # --- Process Video Frames ---
        with tqdm(total=total_frames, desc="Processing frames") as pbar:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                # --- Detect and Track People ---
                results = model.track(frame, classes=[0], conf=conf_threshold, imgsz=imgsz, persist=True)[0]
                detections = Detections(
                    xyxy=results.boxes.xyxy.cpu().numpy(),
                    confidence=results.boxes.conf.cpu().numpy(),
                    class_id=results.boxes.cls.cpu().numpy().astype(int),
                    tracker_id=results.boxes.id.cpu().numpy() if results.boxes.id is not None else None
                )

                # --- Count Crossings ---
                if detections.tracker_id is not None and len(detections.tracker_id) > 0:
                    line_counter.trigger(detections)

                # --- Annotate Frame ---
                labels = [
                    f"ID {track_id if track_id is not None else 'N/A'} {model.names[int(cls_id)]} {conf:.2f}"
                    for xyxy, conf, cls_id, track_id in zip(
                        detections.xyxy,
                        detections.confidence,
                        detections.class_id,
                        detections.tracker_id
                    )
                ]
                frame = box_annotator.annotate(scene=frame, detections=detections)
                frame = label_annotator.annotate(scene=frame, detections=detections, labels=labels)
                frame = line_annotator.annotate(frame=frame, line_counter=line_counter)

                # --- Display Entry/Exit Counts ---
                cv2.rectangle(frame, (20, 20), (300, 80), (0, 0, 0), -1)
                cv2.putText(
                    frame,
                    f"Entry: {line_counter.in_count} | Exit: {line_counter.out_count}",
                    (30, 50),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.8,
                    (0, 255, 0),
                    2
                )

                # --- Crowd Warning ---
                if (line_counter.in_count - line_counter.out_count) > crowd_threshold:
                    cv2.putText(
                        frame,
                        "Crowd Warning!",
                        (30, 70),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.8,
                        (0, 0, 255),
                        2
                    )

                # --- Write Frame ---
                writer.write(frame)

                # --- Save Count Data ---
                count_data.append({
                    "Frame": frame_count,
                    "Entry": line_counter.in_count,
                    "Exit": line_counter.out_count
                })
                frame_count += 1
                pbar.update(1)

        # --- Save Count Data to CSV ---
        pd.DataFrame(count_data).to_csv(output_csv, index=False)
        print("\nCount data saved to counts.csv")

        # --- Download Output Files ---
        if os.path.exists(output_video) and os.path.exists(output_csv):
            print("Processing complete. Downloading files...")
            files.download(output_video)
            files.download(output_csv)
        else:
            print("Error: Output files were not created successfully")

    except Exception as e:
        print(f"Error: {str(e)}")
    finally:
        if 'cap' in locals() and cap.isOpened():
            cap.release()
        if 'writer' in locals() and writer.isOpened():
            writer.release()

# **▶️ Run the Function**

In [ ]:
# --- Run the Function ---
if __name__ == "__main__":
    process_video()